In [21]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle

from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split

from tensorflow import keras
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.layers import Input, Dense, Conv2D, MaxPooling2D
from tensorflow.keras.layers import GlobalAveragePooling2D, AveragePooling2D
from tensorflow.keras.layers import ZeroPadding2D, Dropout, Flatten, add, Reshape
from tensorflow.keras.layers import Activation, BatchNormalization
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.utils import plot_model, to_categorical
from tensorflow.keras import backend as K
from tensorflow.keras import initializers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

from skimage.transform import resize
import cv2

from PIL import Image
import os
from shutil import copyfile

# 1. Load data and preprocess

In [22]:
Image_directory = 'data/UCMerced_LandUse/Images'

#load the labels
category_names = [
    'agricultural',
    'forest',
    'mobilehomepark'
]
#define size of data
num_images = 100  # per category
nb_classes = len(category_names)
patch_size = 256  # width and height in pixel
channels = 3
images_orig = []
labels_num = []
#load data
for i in range(len(category_names)):
    for j in range(num_images):
        img_path = os.path.join(Image_directory, category_names[i], category_names[i]+'{:02d}.tif'.format(j))

        #new_image = plt.imread(img_path)
        #resized_image = resize(new_image, (patch_size/4, patch_size/4, channels))
        img = Image.open(img_path)

        resized_img = img.resize((patch_size//4, patch_size//4), Image.ANTIALIAS)

        arr = np.asarray(resized_img)

        images_orig.append(arr)
        labels_num.append([i])
#image normalization
images_orig = np.array(images_orig)
print('Image Original Data Type: %s' % images_orig[0].dtype)
print('Original: Min: %.3f, Max: %.3f' % (images_orig.min(), images_orig.max()))
# convert from integers to floats
images_orig = images_orig.astype('float32')
# normalize to the range 0-1
images_orig /= 255.0
# confirm the normalization
print('\nImage Resized Data Type: %s' % images_orig[0].dtype)
print('Resized: Min: %.3f, Max: %.3f' % (images_orig.min(), images_orig.max()))
labels_num = np.array(labels_num)

# Save numpy datasets
np.save('data/UCMerced_images_orig_MultiClass.npy', images_orig)
np.save('data/UCMerced_labels_num_MultiClass.npy', labels_num)

Image Original Data Type: uint8
Original: Min: 0.000, Max: 255.000

Image Resized Data Type: float32
Resized: Min: 0.000, Max: 1.000


# 2. Split training, validation and testing set

In [ ]:
# One-hot encoding
labels = to_categorical(labels_num, nb_classes)
nb_images = images_orig.shape[0] #total number of images

X_train, X_test, y_train, y_test = train_test_split(images_orig, labels,
                                                    test_size=0.2,
                                                    stratify = labels,
                                                    random_state=2021)

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train,
                                                  test_size=0.25,
                                                  stratify = y_train,
                                                  random_state=2021)
